# Class V - Conic modelling in JuMP

This notebook describes conic modelling in JuMP through a number of examples.

In [2]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\Oscar\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `C:\Users\Oscar\.julia\registries\JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`


## Example 1: minimum bounding ellipse

Given a set of ellipses centered on the origin

$E(A) = \{ u\;|\;u^\top A^{-1} u <= 1 \}$

find a "minimal" ellipse that contains the provided ellipses

We can formulate this as an SDP:

$\begin{align}
     minimize \quad& trace(WX)\\
   subject to \quad& X \ge A_i, \quad i = 1,...,m \\
              &  X \succeq 0
\end{align}$

where $W$ is a positive-definite matrix of weights to choose between different solutions.

In [96]:
using JuMP, SCS, Plots, LinearAlgebra, Interact

function draw_ellipse(A::Matrix, args...; kwargs...)
    x_values = Float64[]
    y_values = Float64[]
    for angle in 0:0.001π:2π
        u = [cos(angle), sin(angle)]
        z = A * u
        push!(x_values, z[1])
        push!(y_values, z[2])
    end
    plot!(x_values, y_values, args...; kwargs...)
end

function solve_minimum_ellipse_problem(W, A_matrices)
    model = Model(solver = SCSSolver(eps = 1e-6, verbose = false))
    @variable(model, X[1:2, 1:2], SDP)
    @objective(model, Min, tr(W * X))
    for A in A_matrices
        @SDconstraint(model, X >= A)
    end
    status = solve(model)
    return status, JuMP.getvalue(X)
end

solve_minimum_ellipse_problem (generic function with 1 method)

Investigate the model. Here are some things to try:

- What happens if you comment out the first A matrix?

- What happens if you comment out the second A matrix?

- What happens if you comment out the third A matrix?

You can comment lines in Julia using the `#` symbol. As a shortcut, use `[CTRL] + [/]`.

In [97]:
@manipulate for weight in 1:20
    A_matrices = [
        [2.0 0.0; 0.0 1.0],
        [1.0 0.0; 0.0 3.0],
        [2.3896 1.5433; 1.5433 1.35584]
    ]
    W = [1.0 0.0; 0.0 weight]
    status, X_value = solve_minimum_ellipse_problem(W, A_matrices)
    if status == :Optimal
        plot(legend = false)
        draw_ellipse.(A_matrices, color = "gray")
        draw_ellipse(X_value, color="purple", linewidth=2)
    else
        println("Could not solve. Status = $(status)")
    end
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:weight=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:value=>Observable{Int64} with 2 listeners. Value:
10), Observable{Int64} with 2 listeners. Value:
10, Scope("knockout-component-962582f0-07b9-498e-99f9-7919442b140e", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["weight"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-left")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>20,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-center")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-right")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"value"=>(Observable{Int64} with 2 listeners. Value:
10, nothing)), Set(String[]), nothing, Any["knockout"=>"C:\\Users\\Oscar\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout.js", "knockout_punches"=>"C:\\Users\\Oscar\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout_punches.js", "C:\\Users\\Oscar\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\all.js", "C:\\Users\\Oscar\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\style.css", "C:\\Users\\Oscar\\.julia\\packages\\InteractBulma\\Ohu5Y\\src\\..\\assets\\main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":10}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-962582f0-07b9-498e-99f9-7919442b140e\",\"id\":\"ob_92\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"value\"].subscribe((function (val){!(this.valueFromJulia[\"value\"]) ? (WebIO.setval({\"name\":\"value\",\"scope\":\"knockout-component-962582f0-07b9-498e-99f9-7919442b140e\",\"id\":\"ob_91\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJ

# Example 2: clustering points

See Section 2.3 of:
http://www.optimization-online.org/DB_FILE/2005/04/1114.pdf

Given 2-D data pairs $d_i$, $i=1,\ldots,N$, these points can be partitioned into $k$ clusters by solving the following SDP.

$\begin{align}
     minimize \quad& trace(W * (\mathbf{I} - X))\\
   subject\ to \quad& \Sigma_j X_{ij} = 1,\quad i=1, \ldots,N \\
              & trace(X) = k \\
              &  X \succeq 0
\end{align}$,

where $W_{ij} = e^{\frac{-||d_i - d_j||}{\sigma}}$.

In [204]:
""""
    calculate_weight_matrix(data::Matrix{Float64})

Calculates the distance between a list of 2-D data points given as
rows in `data` matrix.
"""
function calculate_weight_matrix(data::Matrix{Float64}, σ = 1.0)
    num_points = size(data, 1)
    W = zeros(num_points, num_points)
    for i in 1:num_points
        for j in (i+1):num_points
            dist = exp(-norm(data[i, :] - data[j, :]) / σ)
            W[i, j] = dist
            W[j, i] = dist
        end
    end
    return W
end

function eye(x, y)
    z = zeros(x, y)
    for i in 1:min(x, y)
        z[i, i] = 1.0
    end
    return z
end

function solve_cluster_problem(data::Matrix{Float64}, num_clusters::Int)
    W = calculate_weight_matrix(data)
    num_points = size(data, 1)
    
    model = Model(solver = SCSSolver(verbose = false))

    @variable(model, X[1:num_points, 1:num_points], SDP)
    @objective(model, Min, tr(W * (eye(num_points, num_points) - X)))
    @constraints(model, begin
        X .>= 0
        [i in 1:num_points], sum(X[i, :]) == 1
        tr(X) == num_clusters
    end)

    status = solve(model)

    X_value = getvalue(X)

    cluster = zeros(Int, num_points)
    cluster_counter = 0
    for i in 1:num_points
        if cluster[i] == 0
            cluster_counter += 1
            cluster[i] = cluster_counter
            for j in (i+1):num_points
                if norm(X_value[i, j] - X_value[i, i]) <= 1e-6
                    cluster[j] = cluster[i]
                end
            end
        end
    end
    return cluster
end

solve_cluster_problem (generic function with 1 method)

Investigate the model. What goes wrong when?

In [205]:
data = vcat(
    rand(Float64, (10, 2)) .+ [2 3],
    rand(Float64, (10, 2)) .+ [4 6],
    rand(Float64, (10, 2)) .+ [3.5 3]
)

@manipulate for num_clusters = 1:4
    which_clusters = solve_cluster_problem(data, num_clusters)
    plot(
        xlabel = "x", ylabel = "y", 
        xlims=(0, 8), ylims = (0, 8),
        legend = :bottomright
    )
    for k in 1:maximum(which_clusters)
        points = which_clusters .== k
        scatter!(data[points, 1], data[points, 2], 
            label = "Cluster $(k)", markersize=10)
    end
    plot!()
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:num_clusters=>Widget{:slider,Int64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:value=>Observable{Int64} with 2 listeners. Value:
2), Observable{Int64} with 2 listeners. Value:
2, Scope("knockout-component-8d34926e-dd84-4472-8b75-e544a34ee41d", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["num_clusters"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-left")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>4,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-center")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-right")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"value"=>(Observable{Int64} with 2 listeners. Value:
2, nothing)), Set(String[]), nothing, Any["knockout"=>"C:\\Users\\Oscar\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout.js", "knockout_punches"=>"C:\\Users\\Oscar\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout_punches.js", "C:\\Users\\Oscar\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\all.js", "C:\\Users\\Oscar\\.julia\\packages\\InteractBase\\3SqBl\\src\\..\\assets\\style.css", "C:\\Users\\Oscar\\.julia\\packages\\InteractBulma\\Ohu5Y\\src\\..\\assets\\main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":2}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-8d34926e-dd84-4472-8b75-e544a34ee41d\",\"id\":\"ob_227\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"value\"].subscribe((function (val){!(this.valueFromJulia[\"value\"]) ? (WebIO.setval({\"name\":\"value\",\"scope\":\"knockout-component-8d34926e-dd84-4472-8b75-e544a34ee41d\",\"id\":\"ob_226\",\"type\":\"observable\"},val)) : undefined; return this.v